# Chapter 24: Class Metaprogramming

## Classes as Objects

## `type`: The Built-In Class Factory

In [5]:
class MySuperClass:
    pass

class MyMixin:
    pass

class MyClass(MySuperClass, MyMixin):
    x = 42
    
    def x2(self):
        return self.x * 2
    
obj = MyClass()
print(obj.x)
print(obj.x2())

42
84


In [4]:
MyClass2 = type('MyClass',
                (MySuperClass, MyMixin),
                {'x': 42, 'x2': lambda self:self.x*2},
                )

obj2 = MyClass2()
print(obj2.x)
print(obj2.x2())

42
84


In [ ]:
monkey = {'name': 'Monkey', 
        'attrs': ['age', 'weight','food']}

# How to create a Monkey class from this dict?
# attributes should not have default values and they'll
# be initialed in __init__ method.

# Use a class factory function.

## A Class Factory Function

In [10]:
# tag::RECORD_FACTORY[]
from typing import Union, Any
from collections.abc import Iterable, Iterator

FieldNames = Union[str, Iterable[str]]  # <1>

def record_factory(cls_name: str, field_names: FieldNames) -> type[tuple]:  # <2>

    slots = parse_identifiers(field_names)  # <3>

    def __init__(self, *args, **kwargs) -> None:  # <4>
        attrs = dict(zip(self.__slots__, args))
        attrs.update(kwargs)
        for name, value in attrs.items():
            setattr(self, name, value)

    def __iter__(self) -> Iterator[Any]:  # <5>
        for name in self.__slots__:
            yield getattr(self, name)

    def __repr__(self):  # <6>
        values = ', '.join(f'{name}={value!r}'
            for name, value in zip(self.__slots__, self))
        cls_name = self.__class__.__name__
        return f'{cls_name}({values})'

    cls_attrs = dict(  # <7>
        __slots__=slots,
        __init__=__init__,
        __iter__=__iter__,
        __repr__=__repr__,
    )

    return type(cls_name, (object,), cls_attrs)  # <8>


def parse_identifiers(names: FieldNames) -> tuple[str, ...]:
    if isinstance(names, str):
        names = names.replace(',', ' ').split()  # <9>
    if not all(s.isidentifier() for s in names):
        raise ValueError('names must all be valid identifiers')
    return tuple(names)
# end::RECORD_FACTORY[]

In [17]:
Dog = record_factory('Dog', 'name weight owner')
rex = Dog('Rex', 30, 'Bob')
rex

Dog(name='Rex', weight=30, owner='Bob')

In [18]:
monkey = {'name': 'Monkey', 
        'attrs': ['age', 'weight','food']}

Monkey = record_factory(monkey['name'], monkey['attrs'])
leo = Monkey(3, 30, 'banana')
print(leo)

Monkey(age=3, weight=30, food='banana')


## Introducing `__init__subclass__`

## Enhancing Classes with a Class Decorator

A class decorator is a callable similarly to a function decorator: it gets the decorated class as an argument, and should return a class to replace the decorated class. Class decorators often return the decorated class itself, after injecting more methods in it via attribute assignment.

The most common reason to choose a class decorator over the simpler `__init_subclass__` is to avoid interfering with other class features, such as inheritance and metaclasses.

## What Happens When: Import Time Versus Runtime

At import time, the interpreter:

1. Parses the source code of a .py module in one pass from top to bottom. This is when a `SyntaxError` may occur.
2. Compiles the bytecode to be executed.
3. Executes the top-level code of the compiled module.

Although parsing and compiling are definitely "import time" activities, other things may happen at that time, because almost every statement in Python is executable in the sense that they can potentially run user code and may change the state of the user program.

In particular, the `import` statement is not merely a declaration, but it actually runs all the top-level code of a module when it is imported for the first time in the process. Further imports of the same module will use a cache, and then the only effect will be binding the imported objects to names in the client module. That top-level code may do anything, including actions typical of "runtime", such as writing to a log or connecting to a database. That's why the border between "import time" and "runtime" is fuzzy: the `import` statement can trigger all sorts of "runtime" behavior. Conversely, "import time" can also happen deep inside runtime, because the `import` statement and the `__import__()` built-in can be used inside any regular function.

### Evaluation Time Experiments

In [1]:
# Example 24-10 builderlib.py: top of the module

print('@ builderlib module start')

class Builder:
    print('@ Builder body')
    
    def __init_subclass__(cls) -> None:
        print(f'@ Builder.__init_subclass__({cls!r})')
        
        def inner_0(self):
            print(f'@ SuperA.__init__subclass__:inner_0({self!r})')
            
        cls.method_a = inner_0
            
    def __init__(self) -> None:
        super().__init__()
        print(f'@ Builder.__init__({self!r})')
        
    def deco(cls):
        print(f'@ deco({cls!r})')
        
        def inner_1(self):
            print(f'@ deco:inner_1({self!r})')
        
        cls.method_b = inner_1
        return cls

# Example 24-11 builderlib.py: bottom of the module

class Descriptor:
    print('@ Descriptor body')
    
    def __init__(self) -> None:
        print(f'@ Descriptor.__init__({self!r})')
        
    def __set_name__(self, owner, name):
        args = (self, owner, name)
        print(f'@ Descriptor.__set_name__({args!r})')
        
    def __set__(self, instance, value):
        args = (self, instance, value)
        print(f'@ Descriptor.__set__({args!r})')
        
    def __repr__(self) -> str:
        return '<Descriptor instance>'
    
print('@ builderlib module end')

@ builderlib module start
@ Builder body
@ Descriptor body
@ builderlib module end


In [ ]:
# Example 24-12. evaldemo.py: script to experiment with builderlib.py

